In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

import keras as keras

from keras import models
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LeakyReLU, BatchNormalization
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelBinarizer
import math
from keras import backend as K

Using TensorFlow backend.


In [2]:
#tf.enable_eager_execution()

In [3]:
#labels = [[0.0, 1.0, 2.0]]
ratio_0_red = 1599 / 744.0
ratio_1_red = 1599 / 638.0
ratio_2_red = 1599 / 217.0
ratio_0_white = 4898 / 1640.0
ratio_1_white = 4898 / 2198.0
ratio_2_white = 4898 / 1060.0

class_weight_white = {"0": ratio_0_white, "1": ratio_1_white, "2": ratio_2_white}
class_weight_red = {"0": ratio_0_red, "1": ratio_1_red, "2": ratio_2_red}

#class_weight_white = [[0, ratio_0_white][1, ratio_1_white][2, ratio_2_white]]
#class_weight_red = [[0, ratio_0_red][1, ratio_1_red][2, ratio_2_red]]
#class_weight = tf.constant([[ratio_0_red, ratio_1_red, ratio_2_red]])
#logits = [0] # shape [batch_size, 3]
#weight_per_label = tf.transpose( tf.matmul(labels
#                           , tf.transpose(class_weight)) ) #shape [1, batch_size]
#labels = [[0,1,2]]
#xent = tf.multiply(weight_per_label, 
#                   tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=labels, name="xent_raw")) #shape [1, batch_size]
#shape 1


logits = ... # shape: [batch_size, 4]
scaled_logits = tf.multiply(logits, class_weight) # shape [batch_size, 4]
loss = tf.nn.softmax_cross_entropy_with_logits(logits=scaled_logits, labels=labels)

In [4]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [5]:
def confusion_matrix_report(y_true, y_pred):    
    cm, labels = confusion_matrix(y_true, y_pred), unique_labels(y_true, y_pred)
    column_width = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    report = " " * column_width + " " + "{:_^{}}".format("Prediction", column_width * len(labels))+ "\n"
    report += " " * column_width + " ".join(["{:>{}}".format(label, column_width) for label in labels]) + "\n"
    for i, label1 in enumerate(labels):
        report += "{:>{}}".format(label1, column_width) + " ".join(["{:{}d}".format(cm[i, j], column_width) for j in range(len(labels))]) + "\n"
    return report

In [6]:
red = pd.read_csv('data/winequality-red1.csv')
white = pd.read_csv('data/winequality-white1.csv')

In [7]:
conditions = [(white['quality'] <= 5),
              (white['quality'] >= 7)]
choices = [0, 2]
white['quality_class'] = np.select(conditions, choices, default = 1)
conditions = [(red['quality'] <= 5),
              (red['quality'] >= 7)]
red['quality_class'] = np.select(conditions, choices, default = 1)

In [8]:
for column in white:
    if column != 'quality' and column != 'quality_class':
        mean = white[column].mean()
        sd = white[column].std()
        white[column] = (white[column]-mean)/sd
for column in red:
    if column != 'quality' and column != 'quality_class':
        mean = red[column].mean()
        sd = red[column].std()
        red[column] = (red[column]-mean)/sd

In [9]:
white_data = white.drop(['quality', 'quality_class'], axis = 1)
red_data = red.drop(['quality', 'quality_class'], axis = 1)

In [10]:
white_class = white['quality_class']
red_class = red['quality_class']

In [11]:
# Data and classes are prepared

print(white_data.describe())
print(white_class.describe())
print(red_data.describe())
print(red_class.describe())

In [12]:
white_data = white_data.values
white_class = white_class.values
#categorical_labels_white = to_categorical(white_class, num_classes=None)
red_data = red_data.values
red_class = red_class.values
#categorical_labels_red = to_categorical(red_class, num_classes=None)
number_of_features = 11

In [16]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

## Structured Approach

## Cross Validation

### Verfication for new approach

#### RELU 8 red

In [30]:
def create_network_relu_8():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(8, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [31]:
neural_network = KerasClassifier(build_fn=create_network_relu_8, 
                                     epochs=75,
                                     verbose=0)

In [32]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [33]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6311687442672026
acc: 
0.6353971232020013
      __Prediction___
         0     1     2
    0  581   157     6
    1  221   357    60
    2   12   127    78

              precision    recall  f1-score   support

           0       0.71      0.78      0.75       744
           1       0.56      0.56      0.56       638
           2       0.54      0.36      0.43       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.60      0.57      0.58      1599
weighted avg       0.63      0.64      0.63      1599



In [34]:
neural_network = KerasClassifier(build_fn=create_network_relu_8, 
                                     epochs=100,
                                     verbose=0)

In [35]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [36]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6361036805684829
acc: 
0.6404002501563477
      __Prediction___
         0     1     2
    0  575   158    11
    1  219   359    60
    2   13   114    90

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       744
           1       0.57      0.56      0.57       638
           2       0.56      0.41      0.48       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.58      0.59      1599
weighted avg       0.63      0.64      0.64      1599



In [37]:
neural_network = KerasClassifier(build_fn=create_network_relu_8, 
                                     epochs=100,
                                     verbose=0)

In [38]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [39]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6444063447939946
acc: 
0.6353971232020013
      __Prediction___
         0     1     2
    0  573   158    13
    1  221   355    62
    2   14   115    88

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       744
           1       0.57      0.56      0.56       638
           2       0.54      0.41      0.46       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.60      0.58      0.59      1599
weighted avg       0.63      0.64      0.63      1599



#### RELU 16 Red

In [40]:
def create_network_relu_16():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [43]:
neural_network = KerasClassifier(build_fn=create_network_relu_16, 
                                     epochs=75,
                                     verbose=0)

In [44]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [45]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6370860806605
acc: 
0.6466541588492808
      __Prediction___
         0     1     2
    0  585   145    14
    1  218   359    61
    2   14   113    90

              precision    recall  f1-score   support

           0       0.72      0.79      0.75       744
           1       0.58      0.56      0.57       638
           2       0.55      0.41      0.47       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.61      0.59      0.60      1599
weighted avg       0.64      0.65      0.64      1599



In [46]:
neural_network = KerasClassifier(build_fn=create_network_relu_16, 
                                     epochs=75,
                                     verbose=0)

In [47]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [48]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6516443817375827
acc: 
0.6353971232020013
      __Prediction___
         0     1     2
    0  570   162    12
    1  223   360    55
    2   20   111    86

              precision    recall  f1-score   support

           0       0.70      0.77      0.73       744
           1       0.57      0.56      0.57       638
           2       0.56      0.40      0.46       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.58      0.59      1599
weighted avg       0.63      0.64      0.63      1599



In [49]:
neural_network = KerasClassifier(build_fn=create_network_relu_16, 
                                     epochs=75,
                                     verbose=0)

In [50]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [51]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.622187229130073
acc: 
0.6541588492808005
      __Prediction___
         0     1     2
    0  581   156     7
    1  210   375    53
    2   15   112    90

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       744
           1       0.58      0.59      0.59       638
           2       0.60      0.41      0.49       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.63      0.59      0.61      1599
weighted avg       0.65      0.65      0.65      1599



#### RELU 32 Red

In [55]:
def create_network_relu_32():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [56]:
neural_network = KerasClassifier(build_fn=create_network_relu_32, 
                                     epochs=75,
                                     verbose=0)

In [57]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [58]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6395354787013959
acc: 
0.6472795497185742
      __Prediction___
         0     1     2
    0  574   156    14
    1  216   366    56
    2   16   106    95

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       744
           1       0.58      0.57      0.58       638
           2       0.58      0.44      0.50       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.62      0.59      0.61      1599
weighted avg       0.64      0.65      0.64      1599



In [59]:
neural_network = KerasClassifier(build_fn=create_network_relu_32, 
                                     epochs=75,
                                     verbose=0)

In [60]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [61]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6410006249027402
acc: 
0.6397748592870544
      __Prediction___
         0     1     2
    0  573   161    10
    1  209   362    67
    2   17   112    88

              precision    recall  f1-score   support

           0       0.72      0.77      0.74       744
           1       0.57      0.57      0.57       638
           2       0.53      0.41      0.46       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.58      0.59      1599
weighted avg       0.63      0.64      0.64      1599



In [62]:
neural_network = KerasClassifier(build_fn=create_network_relu_32, 
                                     epochs=75,
                                     verbose=0)

In [63]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [64]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6266940768159515
acc: 
0.6541588492808005
      __Prediction___
         0     1     2
    0  584   152     8
    1  210   370    58
    2   17   108    92

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       744
           1       0.59      0.58      0.58       638
           2       0.58      0.42      0.49       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.63      0.60      0.61      1599
weighted avg       0.65      0.65      0.65      1599



#### Red Deep

In [106]:
def create_network_deep_red():
    model = tf.keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(24, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))

    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))

    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))

    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(12, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [107]:
neural_network = KerasClassifier(build_fn=create_network_deep_red, 
                                     epochs=75,
                                     verbose=0)

In [108]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [109]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6261949177747328
acc: 
0.6810506566604128
      __Prediction___
         0     1     2
    0  571   153    20
    1  178   390    70
    2   19    70   128

              precision    recall  f1-score   support

           0       0.74      0.77      0.76       744
           1       0.64      0.61      0.62       638
           2       0.59      0.59      0.59       217

   micro avg       0.68      0.68      0.68      1599
   macro avg       0.66      0.66      0.66      1599
weighted avg       0.68      0.68      0.68      1599



In [110]:
neural_network = KerasClassifier(build_fn=create_network_deep_red, 
                                     epochs=75,
                                     verbose=0)

In [111]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [112]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.61358398228325
acc: 
0.6797998749218261
      __Prediction___
         0     1     2
    0  562   168    14
    1  173   394    71
    2   16    70   131

              precision    recall  f1-score   support

           0       0.75      0.76      0.75       744
           1       0.62      0.62      0.62       638
           2       0.61      0.60      0.61       217

   micro avg       0.68      0.68      0.68      1599
   macro avg       0.66      0.66      0.66      1599
weighted avg       0.68      0.68      0.68      1599



In [113]:
neural_network = KerasClassifier(build_fn=create_network_deep_red, 
                                     epochs=75,
                                     verbose=0)

In [114]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [115]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6001875879364265
acc: 
0.699812382739212
      __Prediction___
         0     1     2
    0  569   156    19
    1  159   408    71
    2   13    62   142

              precision    recall  f1-score   support

           0       0.77      0.76      0.77       744
           1       0.65      0.64      0.65       638
           2       0.61      0.65      0.63       217

   micro avg       0.70      0.70      0.70      1599
   macro avg       0.68      0.69      0.68      1599
weighted avg       0.70      0.70      0.70      1599



In [116]:
neural_network = KerasClassifier(build_fn=create_network_deep_red, 
                                     epochs=75,
                                     verbose=0)

In [117]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [118]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6017485465880458
acc: 
0.699812382739212
      __Prediction___
         0     1     2
    0  568   159    17
    1  163   410    65
    2   16    60   141

              precision    recall  f1-score   support

           0       0.76      0.76      0.76       744
           1       0.65      0.64      0.65       638
           2       0.63      0.65      0.64       217

   micro avg       0.70      0.70      0.70      1599
   macro avg       0.68      0.69      0.68      1599
weighted avg       0.70      0.70      0.70      1599



In [119]:
neural_network = KerasClassifier(build_fn=create_network_deep_red, 
                                     epochs=75,
                                     verbose=0)

In [120]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

KeyboardInterrupt: 

In [ ]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

#### RELU 8 white

In [65]:
def create_network_relu_8():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(8, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_white)
    
    return model

In [66]:
neural_network = KerasClassifier(build_fn=create_network_relu_8, 
                                     epochs=75,
                                     verbose=0)

In [67]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [68]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6663773701965007
acc: 
0.5939158840342997
      __Prediction___
         0     1     2
    0 1013   604    23
    1  490  1450   258
    2   39   575   446

              precision    recall  f1-score   support

           0       0.66      0.62      0.64      1640
           1       0.55      0.66      0.60      2198
           2       0.61      0.42      0.50      1060

   micro avg       0.59      0.59      0.59      4898
   macro avg       0.61      0.57      0.58      4898
weighted avg       0.60      0.59      0.59      4898



In [69]:
neural_network = KerasClassifier(build_fn=create_network_relu_8, 
                                     epochs=100,
                                     verbose=0)

In [70]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [71]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6676017669481857
acc: 
0.5977950183748468
      __Prediction___
         0     1     2
    0 1037   570    33
    1  489  1444   265
    2   38   575   447

              precision    recall  f1-score   support

           0       0.66      0.63      0.65      1640
           1       0.56      0.66      0.60      2198
           2       0.60      0.42      0.50      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.57      0.58      4898
weighted avg       0.60      0.60      0.59      4898



In [72]:
neural_network = KerasClassifier(build_fn=create_network_relu_8, 
                                     epochs=100,
                                     verbose=0)

In [73]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [74]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6679075154262929
acc: 
0.5937117190690078
      __Prediction___
         0     1     2
    0 1043   570    27
    1  504  1426   268
    2   38   583   439

              precision    recall  f1-score   support

           0       0.66      0.64      0.65      1640
           1       0.55      0.65      0.60      2198
           2       0.60      0.41      0.49      1060

   micro avg       0.59      0.59      0.59      4898
   macro avg       0.60      0.57      0.58      4898
weighted avg       0.60      0.59      0.59      4898



#### RELU 16 white

In [75]:
def create_network_relu_16():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_white)
    
    return model

In [76]:
neural_network = KerasClassifier(build_fn=create_network_relu_16, 
                                     epochs=75,
                                     verbose=0)

In [77]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [78]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6608395121052415
acc: 
0.6031033074724377
      __Prediction___
         0     1     2
    0 1050   568    22
    1  478  1432   288
    2   43   545   472

              precision    recall  f1-score   support

           0       0.67      0.64      0.65      1640
           1       0.56      0.65      0.60      2198
           2       0.60      0.45      0.51      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.58      0.59      4898
weighted avg       0.61      0.60      0.60      4898



In [79]:
neural_network = KerasClassifier(build_fn=create_network_relu_16, 
                                     epochs=75,
                                     verbose=0)

In [80]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [81]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6695866365013083
acc: 
0.5963658636178032
      __Prediction___
         0     1     2
    0 1047   557    36
    1  494  1404   300
    2   37   553   470

              precision    recall  f1-score   support

           0       0.66      0.64      0.65      1640
           1       0.56      0.64      0.60      2198
           2       0.58      0.44      0.50      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.60      0.57      0.58      4898
weighted avg       0.60      0.60      0.59      4898



In [82]:
neural_network = KerasClassifier(build_fn=create_network_relu_16, 
                                     epochs=75,
                                     verbose=0)

In [83]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [84]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6663773701965007
acc: 
0.5957533687219273
      __Prediction___
         0     1     2
    0 1056   557    27
    1  493  1411   294
    2   38   571   451

              precision    recall  f1-score   support

           0       0.67      0.64      0.65      1640
           1       0.56      0.64      0.60      2198
           2       0.58      0.43      0.49      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.60      0.57      0.58      4898
weighted avg       0.60      0.60      0.59      4898



#### RELU 32 white

In [85]:
def create_network_relu_32():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(32, activation=tf.nn.relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_white)
    
    return model

In [86]:
neural_network = KerasClassifier(build_fn=create_network_relu_32, 
                                     epochs=75,
                                     verbose=0)

In [87]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [88]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6656109784878433
acc: 
0.6041241322988975
      __Prediction___
         0     1     2
    0 1044   567    29
    1  481  1424   293
    2   48   521   491

              precision    recall  f1-score   support

           0       0.66      0.64      0.65      1640
           1       0.57      0.65      0.60      2198
           2       0.60      0.46      0.52      1060

   micro avg       0.60      0.60      0.60      4898
   macro avg       0.61      0.58      0.59      4898
weighted avg       0.61      0.60      0.60      4898



In [89]:
neural_network = KerasClassifier(build_fn=create_network_relu_32, 
                                     epochs=75,
                                     verbose=0)

In [90]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [91]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6662241623843376
acc: 
0.6057574520212332
      __Prediction___
         0     1     2
    0 1046   559    35
    1  473  1419   306
    2   46   512   502

              precision    recall  f1-score   support

           0       0.67      0.64      0.65      1640
           1       0.57      0.65      0.61      2198
           2       0.60      0.47      0.53      1060

   micro avg       0.61      0.61      0.61      4898
   macro avg       0.61      0.59      0.60      4898
weighted avg       0.61      0.61      0.60      4898



In [92]:
neural_network = KerasClassifier(build_fn=create_network_relu_32, 
                                     epochs=75,
                                     verbose=0)

In [93]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [94]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6519750437998775
acc: 
0.6135157207023275
      __Prediction___
         0     1     2
    0 1076   538    26
    1  482  1449   267
    2   37   543   480

              precision    recall  f1-score   support

           0       0.67      0.66      0.67      1640
           1       0.57      0.66      0.61      2198
           2       0.62      0.45      0.52      1060

   micro avg       0.61      0.61      0.61      4898
   macro avg       0.62      0.59      0.60      4898
weighted avg       0.62      0.61      0.61      4898



#### Deep White

In [96]:
def create_network_less_deep_white():
    model = tf.keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(24, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))

    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))

    model.add(layers.Dense(96, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.AlphaDropout(0.01))
    model.add(layers.Dense(12, activation='selu', kernel_initializer='lecun_normal', bias_initializer='lecun_normal'))
    model.add(layers.Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_white)
    
    return model

In [97]:
neural_network = KerasClassifier(build_fn=create_network_less_deep_white, 
                                     epochs=75,
                                     verbose=0)

In [98]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [99]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6366062017484062
acc: 
0.6792568395263373
      __Prediction___
         0     1     2
    0 1153   424    63
    1  426  1488   284
    2   75   299   686

              precision    recall  f1-score   support

           0       0.70      0.70      0.70      1640
           1       0.67      0.68      0.67      2198
           2       0.66      0.65      0.66      1060

   micro avg       0.68      0.68      0.68      4898
   macro avg       0.68      0.68      0.68      4898
weighted avg       0.68      0.68      0.68      4898



In [100]:
neural_network = KerasClassifier(build_fn=create_network_less_deep_white, 
                                     epochs=75,
                                     verbose=0)

In [101]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [102]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6107449480662148
acc: 
0.6894650877909351
      __Prediction___
         0     1     2
    0 1181   407    52
    1  446  1492   260
    2   50   306   704

              precision    recall  f1-score   support

           0       0.70      0.72      0.71      1640
           1       0.68      0.68      0.68      2198
           2       0.69      0.66      0.68      1060

   micro avg       0.69      0.69      0.69      4898
   macro avg       0.69      0.69      0.69      4898
weighted avg       0.69      0.69      0.69      4898



In [103]:
neural_network = KerasClassifier(build_fn=create_network_less_deep_white, 
                                     epochs=75,
                                     verbose=0)

In [104]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [105]:
print('rmse: ' )
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))
print('acc: ' )
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

rmse: 
0.6162360225133208
acc: 
0.6906900775826869
      __Prediction___
         0     1     2
    0 1175   405    60
    1  392  1512   294
    2   55   309   696

              precision    recall  f1-score   support

           0       0.72      0.72      0.72      1640
           1       0.68      0.69      0.68      2198
           2       0.66      0.66      0.66      1060

   micro avg       0.69      0.69      0.69      4898
   macro avg       0.69      0.69      0.69      4898
weighted avg       0.69      0.69      0.69      4898



### Old approach

In [14]:
def rsme(targets, outputs):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(targets, outputs))))  

# Create function returning a compiled network
def create_network():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(33, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              class_weight=class_weight_white)
    
    # Return compiled network
    return model



In [89]:
# Create function returning a compiled network
def create_network_2():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [ ]:
# Create function returning a compiled network
def create_network_2():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [97]:
# Create function returning a compiled network
def create_network_8_balanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              class_weight=class_weight_red)
    
    return model

In [98]:
# Create function returning a compiled network
def create_network_16_balanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(16, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              class_weight=class_weight_red)
    
    return model

In [99]:
# Create function returning a compiled network
def create_network_32_balanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              class_weight=class_weight_red)
    
    return model

In [100]:
# Create function returning a compiled network
def create_network_32_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [118]:
# Create function returning a compiled network
def create_network_48_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(48, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [119]:
# Create function returning a compiled network
def create_network_64_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [120]:
# Create function returning a compiled network
def create_network_128_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [131]:
# Create function returning a compiled network
def create_network_32_unb_tan():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation=tf.nn.tanh),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [144]:
# Create function returning a compiled network
def create_network_32_unb_lrelu():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [161]:
# Create function returning a compiled network
def create_network_2_layers_8sig_8sig_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(8, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [157]:
# Create function returning a compiled network
def create_network_2_layers_8lrelu_8lrelu_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(8, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [158]:
# Create function returning a compiled network
def create_network_2_layers_8tan_8tan_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8, activation=tf.nn.tanh),
        tf.keras.layers.Dense(8, activation=tf.nn.tanh),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [159]:
# Create function returning a compiled network
def create_network_2_layers_8lrelu_8tan_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(8, activation=tf.nn.tanh),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [160]:
# Create function returning a compiled network
def create_network_2_layers_8lrelu_8sig_unbalanced():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(8, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])#,
              #class_weight=class_weight_red)
    
    return model

In [93]:
neural_network = KerasClassifier(build_fn=create_network_2, 
                                     epochs=1000,
                                     verbose=0)

## Red

In [94]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [95]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6540392607455786
acc: 
0.6172607879924953
      __Prediction___
         0     1     2
    0  558   175    11
    1  231   343    64
    2   13   118    86

              precision    recall  f1-score   support

           0       0.70      0.75      0.72       744
           1       0.54      0.54      0.54       638
           2       0.53      0.40      0.46       217

   micro avg       0.62      0.62      0.62      1599
   macro avg       0.59      0.56      0.57      1599
weighted avg       0.61      0.62      0.61      1599



In [35]:
# Use train test split for determination of correct number of epochs
# use this https://stackoverflow.com/questions/47079111/create-keras-callback-to-save-model-predictions-and-targets-for-each-batch-durin

In [101]:
neural_network = KerasClassifier(build_fn=create_network_8_balanced, 
                                     epochs=500,
                                     verbose=0)

In [102]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [103]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6414882633337861
acc: 
0.6316447779862414
      __Prediction___
         0     1     2
    0  569   165    10
    1  231   349    58
    2   13   112    92

              precision    recall  f1-score   support

           0       0.70      0.76      0.73       744
           1       0.56      0.55      0.55       638
           2       0.57      0.42      0.49       217

   micro avg       0.63      0.63      0.63      1599
   macro avg       0.61      0.58      0.59      1599
weighted avg       0.63      0.63      0.63      1599



In [104]:
neural_network = KerasClassifier(build_fn=create_network_16_balanced, 
                                     epochs=500,
                                     verbose=0)

In [105]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [106]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6356119111246085
acc: 
0.6447779862414009
      __Prediction___
         0     1     2
    0  580   152    12
    1  223   359    56
    2   14   111    92

              precision    recall  f1-score   support

           0       0.71      0.78      0.74       744
           1       0.58      0.56      0.57       638
           2       0.57      0.42      0.49       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.62      0.59      0.60      1599
weighted avg       0.64      0.64      0.64      1599



In [107]:
neural_network = KerasClassifier(build_fn=create_network_32_balanced, 
                                     epochs=500,
                                     verbose=0)

In [108]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [109]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.623191567522495
acc: 
0.651031894934334
      __Prediction___
         0     1     2
    0  580   154    10
    1  214   368    56
    2   11   113    93

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       744
           1       0.58      0.58      0.58       638
           2       0.58      0.43      0.49       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.63      0.59      0.61      1599
weighted avg       0.65      0.65      0.65      1599



In [110]:
neural_network = KerasClassifier(build_fn=create_network_32_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [111]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [112]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6105185976682542
acc: 
0.6610381488430269
      __Prediction___
         0     1     2
    0  582   154     8
    1  200   378    60
    2   10   110    97

              precision    recall  f1-score   support

           0       0.73      0.78      0.76       744
           1       0.59      0.59      0.59       638
           2       0.59      0.45      0.51       217

   micro avg       0.66      0.66      0.66      1599
   macro avg       0.64      0.61      0.62      1599
weighted avg       0.66      0.66      0.66      1599



In [113]:
neural_network = KerasClassifier(build_fn=create_network_32_unbalanced, 
                                     epochs=250,
                                     verbose=0)

In [114]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [115]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6306731268022636
acc: 
0.6454033771106942
      __Prediction___
         0     1     2
    0  581   153    10
    1  215   363    60
    2   13   116    88

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       744
           1       0.57      0.57      0.57       638
           2       0.56      0.41      0.47       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.62      0.59      0.60      1599
weighted avg       0.64      0.65      0.64      1599



In [121]:
neural_network = KerasClassifier(build_fn=create_network_48_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [122]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [123]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6110305630392845
acc: 
0.6660412757973734
      __Prediction___
         0     1     2
    0  586   150     8
    1  196   389    53
    2   13   114    90

              precision    recall  f1-score   support

           0       0.74      0.79      0.76       744
           1       0.60      0.61      0.60       638
           2       0.60      0.41      0.49       217

   micro avg       0.67      0.67      0.67      1599
   macro avg       0.64      0.60      0.62      1599
weighted avg       0.66      0.67      0.66      1599



In [124]:
neural_network = KerasClassifier(build_fn=create_network_64_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [125]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [126]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6181535578949711
acc: 
0.6591619762351469
      __Prediction___
         0     1     2
    0  577   159     8
    1  195   383    60
    2   14   109    94

              precision    recall  f1-score   support

           0       0.73      0.78      0.75       744
           1       0.59      0.60      0.59       638
           2       0.58      0.43      0.50       217

   micro avg       0.66      0.66      0.66      1599
   macro avg       0.63      0.60      0.61      1599
weighted avg       0.66      0.66      0.66      1599



In [127]:
neural_network = KerasClassifier(build_fn=create_network_128_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [128]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [129]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6156190922898224
acc: 
0.6660412757973734
      __Prediction___
         0     1     2
    0  573   160    11
    1  190   391    57
    2   13   103   101

              precision    recall  f1-score   support

           0       0.74      0.77      0.75       744
           1       0.60      0.61      0.61       638
           2       0.60      0.47      0.52       217

   micro avg       0.67      0.67      0.67      1599
   macro avg       0.64      0.62      0.63      1599
weighted avg       0.66      0.67      0.66      1599



In [132]:
neural_network = KerasClassifier(build_fn=create_network_32_unb_tan, 
                                     epochs=500,
                                     verbose=0)

In [133]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [134]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.650203220264462
acc: 
0.6541588492808005
      __Prediction___
         0     1     2
    0  549   170    25
    1  196   379    63
    2   16    83   118

              precision    recall  f1-score   support

           0       0.72      0.74      0.73       744
           1       0.60      0.59      0.60       638
           2       0.57      0.54      0.56       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.63      0.63      0.63      1599
weighted avg       0.65      0.65      0.65      1599



In [135]:
neural_network = KerasClassifier(build_fn=create_network_32_unb_tan, 
                                     epochs=250,
                                     verbose=0)

In [136]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [137]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6186592050616435
acc: 
0.6585365853658537
      __Prediction___
         0     1     2
    0  569   166     9
    1  203   380    55
    2   13   100   104

              precision    recall  f1-score   support

           0       0.72      0.76      0.74       744
           1       0.59      0.60      0.59       638
           2       0.62      0.48      0.54       217

   micro avg       0.66      0.66      0.66      1599
   macro avg       0.64      0.61      0.63      1599
weighted avg       0.66      0.66      0.66      1599



In [138]:
neural_network = KerasClassifier(build_fn=create_network_32_unb_tan, 
                                     epochs=100,
                                     verbose=0)

In [139]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [140]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.630177119547553
acc: 
0.6516572858036273
      __Prediction___
         0     1     2
    0  581   153    10
    1  209   373    56
    2   16   113    88

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       744
           1       0.58      0.58      0.58       638
           2       0.57      0.41      0.47       217

   micro avg       0.65      0.65      0.65      1599
   macro avg       0.63      0.59      0.60      1599
weighted avg       0.65      0.65      0.65      1599



In [145]:
neural_network = KerasClassifier(build_fn=create_network_32_unb_lrelu, 
                                     epochs=250,
                                     verbose=0)

In [146]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [147]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6375767130633383
acc: 
0.6554096310193871
      __Prediction___
         0     1     2
    0  577   153    14
    1  210   368    60
    2   19    95   103

              precision    recall  f1-score   support

           0       0.72      0.78      0.74       744
           1       0.60      0.58      0.59       638
           2       0.58      0.47      0.52       217

   micro avg       0.66      0.66      0.66      1599
   macro avg       0.63      0.61      0.62      1599
weighted avg       0.65      0.66      0.65      1599



In [148]:
neural_network = KerasClassifier(build_fn=create_network_32_unb_lrelu, 
                                     epochs=500,
                                     verbose=0)

In [149]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [150]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6453761065838611
acc: 
0.6416510318949343
      __Prediction___
         0     1     2
    0  565   166    13
    1  211   359    68
    2   18    97   102

              precision    recall  f1-score   support

           0       0.71      0.76      0.73       744
           1       0.58      0.56      0.57       638
           2       0.56      0.47      0.51       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.62      0.60      0.60      1599
weighted avg       0.64      0.64      0.64      1599



In [151]:
neural_network = KerasClassifier(build_fn=create_network_32_unb_lrelu, 
                                     epochs=50,
                                     verbose=0)

In [152]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [153]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6434351214164253
acc: 
0.6404002501563477
      __Prediction___
         0     1     2
    0  578   155    11
    1  221   363    54
    2   18   116    83

              precision    recall  f1-score   support

           0       0.71      0.78      0.74       744
           1       0.57      0.57      0.57       638
           2       0.56      0.38      0.45       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.58      0.59      1599
weighted avg       0.63      0.64      0.63      1599



In [154]:
neural_network = KerasClassifier(build_fn=create_network_32_unb_lrelu, 
                                     epochs=10,
                                     verbose=0)

In [155]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [156]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6535609869991703
acc: 
0.6216385240775485
      __Prediction___
         0     1     2
    0  582   152    10
    1  246   335    57
    2   16   124    77

              precision    recall  f1-score   support

           0       0.69      0.78      0.73       744
           1       0.55      0.53      0.54       638
           2       0.53      0.35      0.43       217

   micro avg       0.62      0.62      0.62      1599
   macro avg       0.59      0.55      0.57      1599
weighted avg       0.61      0.62      0.61      1599



## 2 Layers

In [167]:
neural_network = KerasClassifier(build_fn=create_network_2_layers_8sig_8sig_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [168]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [169]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6151109460271793
acc: 
0.6572858036272671
      __Prediction___
         0     1     2
    0  583   151    10
    1  215   373    50
    2    9   113    95

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       744
           1       0.59      0.58      0.59       638
           2       0.61      0.44      0.51       217

   micro avg       0.66      0.66      0.66      1599
   macro avg       0.64      0.60      0.62      1599
weighted avg       0.65      0.66      0.65      1599



In [170]:
neural_network = KerasClassifier(build_fn=create_network_2_layers_8tan_8tan_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [171]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [172]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6291839319806762
acc: 
0.6397748592870544
      __Prediction___
         0     1     2
    0  579   152    13
    1  217   349    72
    2    6   116    95

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       744
           1       0.57      0.55      0.56       638
           2       0.53      0.44      0.48       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.59      0.59      1599
weighted avg       0.63      0.64      0.64      1599



In [173]:
neural_network = KerasClassifier(build_fn=create_network_2_layers_8lrelu_8lrelu_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [174]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [175]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6473112716665117
acc: 
0.6391494684177611
      __Prediction___
         0     1     2
    0  563   164    17
    1  215   364    59
    2   14   108    95

              precision    recall  f1-score   support

           0       0.71      0.76      0.73       744
           1       0.57      0.57      0.57       638
           2       0.56      0.44      0.49       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.59      0.60      1599
weighted avg       0.63      0.64      0.64      1599



In [176]:
neural_network = KerasClassifier(build_fn=create_network_2_layers_8lrelu_8sig_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [177]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [178]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6458604414412116
acc: 
0.6372732958098811
      __Prediction___
         0     1     2
    0  566   163    15
    1  217   355    66
    2   14   105    98

              precision    recall  f1-score   support

           0       0.71      0.76      0.73       744
           1       0.57      0.56      0.56       638
           2       0.55      0.45      0.49       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.59      0.60      1599
weighted avg       0.63      0.64      0.63      1599



In [179]:
neural_network = KerasClassifier(build_fn=create_network_2_layers_8lrelu_8tan_unbalanced, 
                                     epochs=500,
                                     verbose=0)

In [180]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

In [181]:
print('rmse: ' )
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))
print('acc: ' )
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

rmse: 
0.6473112716665117
acc: 
0.6410256410256411
      __Prediction___
         0     1     2
    0  566   161    17
    1  214   362    62
    2   15   105    97

              precision    recall  f1-score   support

           0       0.71      0.76      0.74       744
           1       0.58      0.57      0.57       638
           2       0.55      0.45      0.49       217

   micro avg       0.64      0.64      0.64      1599
   macro avg       0.61      0.59      0.60      1599
weighted avg       0.64      0.64      0.64      1599



# KrimsKrams

In [34]:
prediction_red_1000ep = prediction_red # accuracy 65%

In [73]:
prediction_red_500ep = prediction_red # accuracy 66%

In [19]:
prediction_red_1000ep_2 = prediction_red # accuracy 68%

In [41]:
print(math.sqrt(np.mean((red_class-prediction_red_1000ep_2)*(red_class-prediction_red_1000ep_2))))
print(accuracy_score(red_class, prediction_red_1000ep_2))
print(confusion_matrix_report(red_class, prediction_red_1000ep_2))
print(classification_report(red_class, prediction_red_1000ep_2))

0.6216844514890689
0.6791744840525328
      __Prediction___
         0     1     2
    0  524   200    20
    1  142   432    64
    2   15    72   130

              precision    recall  f1-score   support

           0       0.77      0.70      0.74       744
           1       0.61      0.68      0.64       638
           2       0.61      0.60      0.60       217

   micro avg       0.68      0.68      0.68      1599
   macro avg       0.66      0.66      0.66      1599
weighted avg       0.69      0.68      0.68      1599



## White

In [28]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [29]:
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

0.6962025316455697
      __Prediction___
         0     1     2
    0 1177   404    59
    1  401  1517   280
    2   56   288   716

              precision    recall  f1-score   support

           0       0.72      0.72      0.72      1640
           1       0.69      0.69      0.69      2198
           2       0.68      0.68      0.68      1060

   micro avg       0.70      0.70      0.70      4898
   macro avg       0.70      0.69      0.69      4898
weighted avg       0.70      0.70      0.70      4898



In [30]:
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))

0.6117469913127119


In [25]:
prediction_white_1000ep = prediction_white # accuracy 69%

In [44]:
print(math.sqrt(np.mean((white_class-prediction_white_1000ep)*(white_class-prediction_white_1000ep))))
print(accuracy_score(white_class, prediction_white_1000ep))
print(confusion_matrix_report(white_class, prediction_white_1000ep))
print(classification_report(white_class, prediction_white_1000ep))

0.6193754675345038
0.6904859126173949
      __Prediction___
         0     1     2
    0 1153   430    57
    1  402  1548   248
    2   64   315   681

              precision    recall  f1-score   support

           0       0.71      0.70      0.71      1640
           1       0.68      0.70      0.69      2198
           2       0.69      0.64      0.67      1060

   micro avg       0.69      0.69      0.69      4898
   macro avg       0.69      0.68      0.69      4898
weighted avg       0.69      0.69      0.69      4898



In [ ]:
prediction_white_1000ep_deeper = prediction_white # accuracy 68.5%

# Krims Krams

model = tf.keras.Sequential()
model.add(layers.Dense(33, activation='sigmoid'))
model.add(layers.Dense(132, activation='sigmoid'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(33, activation='linear'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mean_squared_error',   # can also use sparse_categorical_crossentropy then you don't need labels
              metrics=['accuracy'])

white_sk_train, white_sk_test = train_test_split(white, test_size=0.2, random_state=42)
red_sk_train, red_sk_test = train_test_split(red, test_size=0.2, random_state=42)

white_data_train = white_sk_train.drop(['quality', 'quality_class'], axis = 1)
white_data_test = white_sk_test.drop(['quality', 'quality_class'], axis = 1)
red_data_train = red_sk_train.drop(['quality', 'quality_class'], axis = 1)
red_data_test = red_sk_test.drop(['quality', 'quality_class'], axis = 1)

white_class_train = white_sk_train['quality_class']
white_class_test = white_sk_test['quality_class']
red_class_train = red_sk_train['quality_class']
red_class_test = red_sk_test['quality_class']

In [30]:
model = tf.keras.Sequential()
model.add(layers.Dense(33, activation='sigmoid'))
model.add(layers.Dense(132, activation='sigmoid'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(33, activation='linear'))
model.add(layers.Dense(10, activation='softmax'))

In [34]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mean_squared_error',   # can also use sparse_categorical_crossentropy then you don't need labels
              metrics=['accuracy'])

In [33]:
model.fit(white_data_train, white_class_train, epochs=100, batch_size=16)

In [ ]:
test_loss, test_acc = model.evaluate(white_data_test, white_class_test)
print('Test accuracy:', test_acc)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(33, activation=tf.nn.relu),
  tf.keras.layers.Dense(132, activation=tf.nn.sigmoid),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(132*4, activation=tf.nn.selu),
  tf.keras.layers.Dense(132*16, activation=tf.nn.sigmoid),
  tf.keras.layers.Dense(132*4, activation=tf.nn.relu),
  tf.keras.layers.Dense(132, activation=tf.nn.sigmoid),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(33, activation=tf.nn.relu),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(white_data_train, white_class_train, epochs=150)
model.evaluate(white_data_test, white_class_test)

In [31]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(33, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(132, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(132, activation=tf.nn.leaky_relu),
    tf.keras.layers.Dense(33),
    tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])

model.epochs = 500
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#model.fit(white_data_train, white_class_train, epochs=1000)
#model.evaluate(white_data_test, white_class_test)


ValueError: optimizer must be an instance of tf.train.Optimizer, not a <class 'str'>

white_data_train = white_data_train.values
white_class_train = white_class_train.values
categorical_labels_train = to_categorical(white_class_train, num_classes=None)
white_data_test = white_data_test.values
white_class_test = white_class_test.values
categorical_labels_test = to_categorical(white_class_test, num_classes=None)

In [ ]:
# Maximum Accuracy bisher 68% (very wide but not so deep network), konsistent 65%, new best is 0.706
# Could use weights for losses for balancing classes (as described by some paper or website)

In [ ]:
#predictions = model.predict(white_data_test)
predictions = model.predict_classes(white_data_test)

In [ ]:
print(accuracy_score(white_class_test, predictions))
print(confusion_matrix_report(white_class_test, predictions))
print(classification_report(white_class_test, predictions))